# Clinical DPO Training - Google Colab

This notebook trains a Llama 3 8B model using Direct Preference Optimization (DPO) on clinical mental health data.

**Hardware**: Use GPU runtime (T4 or better)
**Time**: ~30-60 minutes
**Output**: Trained model ready for download

## Step 1: Check GPU and Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

In [ ]:
# Install required packages
!pip install -q transformers trl peft accelerate datasets bitsandbytes

print("✅ Dependencies installed!")

## Step 2: Upload Your Training Data

**Important**: Upload these files from your Mac:
- `Landing/DPO/Data/dpo_train_dataset.jsonl` (526 pairs)
- `Landing/DPO/Data/dpo_holdout_dataset.jsonl` (59 pairs)

Click the folder icon on the left → Upload → Select both files

In [ ]:
# Verify uploaded files
import os
from pathlib import Path

required_files = ['dpo_train_dataset.jsonl', 'dpo_holdout_dataset.jsonl']
missing_files = [f for f in required_files if not Path(f).exists()]

if missing_files:
    print("❌ Missing files:")
    for f in missing_files:
        print(f"   - {f}")
    print("\n📤 Please upload the files using the file browser on the left")
else:
    print("✅ All training files found!")
    !wc -l *.jsonl

## Step 3: Training Configuration

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset
from datetime import datetime

# Configuration
BASE_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"
OUTPUT_DIR = "./clinical_dpo_model"
TRAIN_DATA = "dpo_train_dataset.jsonl"

# Training hyperparameters
BATCH_SIZE = 4              # T4 GPU can handle larger batches
GRAD_ACCUM_STEPS = 2        # Effective batch size = 8
NUM_EPOCHS = 3
LEARNING_RATE = 5e-6
MAX_SEQ_LENGTH = 2048       # Full length for long responses
BETA = 0.1

# LoRA configuration
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

print("✅ Configuration loaded")

## Step 4: Load Dataset

In [ ]:
print(f"📂 Loading training data...")
dataset = load_dataset("json", data_files=TRAIN_DATA, split="train")

print(f"✅ Loaded {len(dataset)} training examples")
print(f"\n📝 Sample entry:")
sample = dataset[0]
print(f"   Prompt: {sample['prompt'][:100]}...")
print(f"   Chosen: {sample['chosen'][:100]}...")
print(f"   Rejected: {sample['rejected'][:100]}...")

## Step 5: Load Model and Tokenizer

In [ ]:
print(f"🔄 Loading {BASE_MODEL}...")
print("   This may take a few minutes (downloading ~16GB)...\n")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Load model with 4-bit quantization (saves memory)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_4bit=True,  # Use 4-bit quantization
    device_map="auto",
    torch_dtype=torch.float16,
)

print(f"✅ Model loaded successfully")
print(f"   Device: {next(model.parameters()).device}")

## Step 6: Add LoRA Adapters

In [ ]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA adapters configured:")
print(f"   Trainable params: {trainable:,} ({trainable/total*100:.2f}%)")
print(f"   Total params: {total:,}")

## Step 7: Configure DPO Trainer

In [ ]:
training_args = DPOConfig(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    max_length=MAX_SEQ_LENGTH,
    max_prompt_length=512,
    beta=BETA,
    
    # GPU optimization
    bf16=True,  # Use bfloat16 on GPU
    fp16=False,
    
    # Logging and saving
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    
    # Optimization
    optim="adamw_torch",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    
    # Other
    remove_unused_columns=False,
    report_to="none",
    seed=42,
)

print(f"✅ Training configuration ready")
print(f"   Effective batch size: {BATCH_SIZE * GRAD_ACCUM_STEPS}")
print(f"   Total steps: ~{len(dataset) // (BATCH_SIZE * GRAD_ACCUM_STEPS) * NUM_EPOCHS}")

## Step 8: Start Training 🚀

**Expected time**: 30-60 minutes on T4 GPU

You'll see:
- Loss values every 10 steps
- Progress bar showing estimated completion time
- Checkpoints saved every 50 steps

In [ ]:
print("="*70)
print("STARTING DPO TRAINING")
print("="*70)

start_time = datetime.now()
print(f"\n🚀 Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

try:
    trainer = DPOTrainer(
        model=model,
        ref_model=None,
        args=training_args,
        train_dataset=dataset,
        processing_class=tokenizer,
    )
    
    trainer.train()
    
    end_time = datetime.now()
    duration = end_time - start_time
    
    print(f"\n{'='*70}")
    print(f"✅ TRAINING COMPLETED!")
    print(f"{'='*70}")
    print(f"   Duration: {duration}")
    print(f"   Model saved to: {OUTPUT_DIR}")
    
except Exception as e:
    print(f"\n❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

## Step 9: Save Final Model

In [ ]:
final_output = f"{OUTPUT_DIR}/final_model"

print(f"💾 Saving final model to {final_output}...")
trainer.model.save_pretrained(final_output)
tokenizer.save_pretrained(final_output)

print(f"\n✅ Model saved!")
print(f"\n📁 Files created:")
!ls -lh {final_output}

## Step 10: Download Trained Model

Zip the model for easy download:

In [ ]:
import shutil

print("📦 Creating zip file...")
shutil.make_archive('clinical_dpo_model', 'zip', OUTPUT_DIR, 'final_model')

print("\n✅ Model packaged!")
print("\n📥 Download the file:")
print("   1. Click the folder icon on the left")
print("   2. Find 'clinical_dpo_model.zip'")
print("   3. Right-click → Download")
print("\n   Or run this to download:")

from google.colab import files
files.download('clinical_dpo_model.zip')

## Step 11: Quick Inference Test (Optional)

In [ ]:
# Test the trained model
test_prompt = "I've been feeling really anxious about my upcoming presentation at work."

formatted_prompt = f"<|user|>\n{test_prompt}\n<|assistant|>\n"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

print(f"🧪 Testing trained model...\n")
print(f"Patient: {test_prompt}\n")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.split("<|assistant|>")[-1].strip()

print(f"Therapist: {response}")

## ✅ Training Complete!

### Next Steps:

1. **Download the model**: The zip file contains your trained LoRA adapters
2. **Transfer to Mac**: Unzip in `Landing/Training/outputs/final_model/`
3. **Run evaluation**: Use your Mac to run `evaluate_model.py`
4. **Test inference**: Run `inference_test.py` with sample prompts

### Model Usage on Mac:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "./outputs/final_model",
    device_map="auto",
    torch_dtype=torch.float16
)
```